In [ ]:
import os
from google.colab import auth
import gspread
from google.auth import default
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.api.types import is_string_dtype, is_numeric_dtype

Preprocessing

In [ ]:
# authenticate
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# open worksheet
worksheet = gc.open('Combined_19-21').sheet1

# get_all_values gives a list of rows
rows = worksheet.get_all_values()

# convert to a DataFrame and render
df = pd.DataFrame(rows)
df.columns = df.iloc[0]
df = df.iloc[1:]
df.head()

In [ ]:
# get rid of ',' and '$'
df['Cost'] = df['Cost'].str.replace(',', '', regex=True)
df['Cost'] = df['Cost'].str.replace('$', '', regex=True)

In [ ]:
# define data types for columns
new_df_schema = {
'Location': df['Location'].astype(str),
'Date': pd.to_datetime(df['Date']),
'Description': df['Description'].astype(str),
'Cost': pd.to_numeric(df['Cost'], errors='coerce'),
'Category': df['Category'].astype(str),
}

# update the data types
df = pd.DataFrame(new_df_schema)

In [ ]:
# change tax to misc
df['Category'] = df['Category'].str.replace('tax', 'misc')

In [ ]:
# change clothing to clothes
df['Category'] = df['Category'].str.replace('clothing', 'clothes')

In [ ]:
# change coupons to grocery
df['Category'] = df['Category'].str.replace('coupons', 'grocery')

In [ ]:
# change all descriptions to lowercase
df['Description'] = df['Description'].str.lower()

In [ ]:
df['Location'] = df['Location'].str.replace(" ", "")

In [ ]:
df.info()

In [ ]:
df.describe(include='all', datetime_is_numeric=True)

In [ ]:
df.isnull().sum()

In [ ]:
# view rows containing NaN
df1 = df[df.isna().any(axis=1)]
print(df1)

In [ ]:
# change Cost NaN to 0
df['Cost'] = df['Cost'].fillna(0)

In [ ]:
df.to_csv('expenses_data.csv', index=False)

In [ ]:
# new dataframe without negatives or zeros
df_pos = df.copy()

In [ ]:
# get names of indexes for which column Cost is <= 0
indexNames = df_pos[df_pos['Cost'] <= 0].index
# delete these row indexes from dataFrame
df_pos.drop(indexNames , inplace=True)

In [ ]:
# change subcategories to grocery
df_pos['Category'] = df_pos['Category'].replace(dict.fromkeys(['bread','breakfast','canned','condiments','dairy','grains','meat','pasta','produce','snacks'], 'grocery'))

In [ ]:
# split Date into Day, Month, Year
df_pos['Day'] = df_pos['Date'].dt.day.apply(str)
df_pos['Month'] = df_pos['Date'].dt.month_name()
df_pos['Year'] = df_pos['Date'].dt.year.apply(str)
df_pos = df_pos.drop('Date', axis=1)

In [ ]:
df_pos.to_csv('pos_expenses_data.csv', index=False)

Exploratory Data Analysis

In [ ]:
df_pos.info()

In [ ]:
df_pos.describe(include='all')

In [ ]:
# create numeric list, category list and small cat list of categories with <15 unique entries
num_list = []
cat_list = []
small_cat_list = []

for column in df_pos:
  if is_numeric_dtype(df_pos[column]):
    num_list.append(column)
  elif is_string_dtype(df_pos[column]):
    cat_list.append(column)

for column in df_pos:
  if len(pd.unique(df_pos[column])) < 15:
    small_cat_list.append(column)

print(num_list)
print(cat_list)
print(small_cat_list)

In [ ]:
# bar chart for frequency of categorical variables, histogram for continuous
for column in df_pos:
  plt.figure(column)
  plt.title(column)
  if is_numeric_dtype(df_pos[column]):
    df_pos[column].plot(kind='hist')
  else:
    df_pos[column].value_counts()[:10].plot(kind='bar')

In [ ]:
# bar plot of top 10 by values for each variable by cost 
for column in df_pos:
  if column != 'Cost':
    sum_cost =  df_pos.groupby([column]).sum().reset_index()
    sum_cost = sum_cost.sort_values(by=['Cost'], ascending=False)
    sum_cost[:10].plot(x=column, y='Cost', kind='bar', title='Total Cost by %s' % column)

In [ ]:
# grouped bar charts of categorical variables
for i in range(0, len(cat_list)):
  primary_cat = cat_list[i]
  for j in range(0, len(small_cat_list)):
    secondary_cat = small_cat_list[j]
    if secondary_cat != primary_cat:
      plt.figure (figsize = (15,15))
      chart = sns.countplot(
        data = df_pos,
        x = primary_cat, 
        hue = secondary_cat,
        palette = 'YlGnBu',
        order = df_pos[primary_cat].value_counts().iloc[:10].index #show only TOP10
      )

In [ ]:
#for i in range(0, len(cat_list)):
#  primary_cat = cat_list[i]
#  for j in range(0, len(small_cat_list)):
#    secondary_cat = small_cat_list[j]
#    if secondary_cat != primary_cat:
#      df_group = df_pos.groupby([primary_cat, secondary_cat]).sum().reset_index()
#      df_unique = df_group.loc[df_group.groupby([primary_cat, secondary_cat])['Cost'].idxmax()]
#      df_sort = df_unique.sort_values(by=['Cost'], ascending=False)
#      print(df_sort)

In [ ]:
# strip plot
for i in range(0, len(cat_list)):
  primary_cat = cat_list[i]
  for j in range(0, len(small_cat_list)):
    secondary_cat = small_cat_list[j]
    if secondary_cat != primary_cat:
      plt.figure (figsize = (20,10))
      df_group = df_pos.groupby([primary_cat, secondary_cat]).sum().reset_index()
      df_unique = df_group.loc[df_group.groupby([primary_cat])['Cost'].idxmax()]
      df_sort = df_unique.sort_values(by=['Cost'], ascending=False)
      chart = sns.stripplot(
        data = df_sort,
        x = primary_cat,
        y = 'Cost',
        hue = secondary_cat,
        palette = 'Paired',
        order = df_sort[primary_cat][:10] #show only TOP10
      )

In [ ]:
# box plot
for i in range(0, len(small_cat_list)):
  cat = small_cat_list[i]
  num = df_pos['Cost']
  plt.figure (figsize = (15,15))
  sns.boxplot( x = cat, y = num, data = df_pos, palette = "GnBu")